In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df = pd.read_csv('C://Users//HP//Downloads//Data_for_UCI_named.csv')

df.head(5)

,tau1,tau2,tau3,tau4,p1,p2,p3,p4,g1,g2,g3,g4,stab,stabf
0,2.959060,3.079885,8.381025,9.780754,3.763085,-0.782604,-1.257395,-1.723086,0.650456,0.859578,0.887445,0.958034,0.055347,unstable
1,9.304097,4.902524,3.047541,1.369357,5.067812,-1.940058,-1.872742,-1.255012,0.413441,0.862414,0.562139,0.781760,-0.005957,stable
2,8.971707,8.848428,3.046479,1.214518,3.405158,-1.207456,-1.277210,-0.920492,0.163041,0.766689,0.839444,0.109853,0.003471,unstable
3,0.716415,7.669600,4.486641,2.340563,3.963791,-1.027473,-1.938944,-0.997374,0.446209,0.976744,0.929381,0.362718,0.028871,unstable
4,3.134112,7.608772,4.943759,9.857573,3.525811,-1.125531,-1.845975,-0.554305,0.797110,0.455450,0.656947,0.820923,0.049860,unstable


In [17]:
df = df.drop(columns ='stab')
df.isna().sum()

tau1     0
tau2     0
tau3     0
tau4     0
p1       0
p2       0
p3       0
p4       0
g1       0
g2       0
g3       0
g4       0
stabf    0
dtype: int64

In [18]:
df_stab = df[df.stabf == 'stable'].sample(1000)
df_unstab = df[df.stabf == 'unstable'].sample(1000)

data_df = df_stab.append(df_unstab)

data_df.isna().sum()

tau1     0
tau2     0
tau3     0
tau4     0
p1       0
p2       0
p3       0
p4       0
g1       0
g2       0
g3       0
g4       0
stabf    0
dtype: int64

In [19]:
import sklearn.utils
df = sklearn.utils.shuffle(data_df)
df = data_df.reset_index(drop=True)
df.shape

df.stabf.value_counts()

stable      1000
unstable    1000
Name: stabf, dtype: int64

In [20]:
#more processing

X = data_df.drop(columns='stabf')
y = data_df['stabf']

#splits data into training and test sets
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
y_train.value_counts()

stable      805
unstable    795
Name: stabf, dtype: int64

In [21]:
import imblearn
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=1)
x_train_balanced, y_balanced = smote.fit_resample(x_train, y_train)

In [22]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
normalised_train_df = scaler.fit_transform(x_train_balanced)
normalised_train_df = pd.DataFrame(normalised_train_df, columns=x_train_balanced.columns)

In [23]:
x_test = x_test.reset_index(drop=True)
normalised_test_df = scaler.transform(x_test)
normalised_test_df = pd.DataFrame(normalised_test_df, columns=x_test.columns)

In [24]:
#Logistic Regression
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression()
log_reg.fit(normalised_train_df, y_balanced)

LogisticRegression()

In [25]:
LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                  max_iter=100, multi_class='auto', n_jobs=None, penalty='12', random_state=1, solver='lbfgs', 
                  tol=0.0001, verbose=0,warm_start=False)

LogisticRegression(penalty='12', random_state=1)

In [26]:
# Cross Validation and Accuracy
from sklearn.model_selection import cross_val_score
scores = cross_val_score(log_reg, normalised_train_df, y_balanced, cv=5, scoring='f1_macro')
scores

array([0.74485931, 0.7919074 , 0.81662693, 0.79187528, 0.77010651])

In [27]:
#Confusion Matrix
from sklearn.metrics import recall_score, accuracy_score, precision_score, f1_score, confusion_matrix

new_predictions = log_reg.predict(normalised_test_df)

cnf_mat = confusion_matrix(y_true=y_test, y_pred=new_predictions, labels=['stable','unstable'])
cnf_mat

array([[156,  39],
       [ 46, 159]], dtype=int64)

In [28]:
accuracy = accuracy_score(y_true=y_test, y_pred=new_predictions)
print('Accuracy: {}'.format(round(accuracy*100), 2))

Accuracy: 79


In [29]:
precision = precision_score(y_true=y_test, y_pred=new_predictions, pos_label='stable')
print('Precision: {}'.format(round(precision*100), 2))

Precision: 77


In [30]:
recall = recall_score(y_true=y_test, y_pred=new_predictions, pos_label='stable')
print('Recall: {}'.format(round(accuracy*100), 2))

Recall: 79


In [31]:
f1 = f1_score(y_true=y_test, y_pred=new_predictions, pos_label='stable')
print('F1: {}'.format(round(f1*100), 2))

F1: 79


In [32]:
from sklearn.tree import DecisionTreeClassifier

dec_tree = DecisionTreeClassifier()
dec_tree.fit(normalised_train_df, y_balanced)

dec_tree.predict(normalised_test_df)

array(['stable', 'unstable', 'unstable', 'unstable', 'unstable',
       'unstable', 'stable', 'unstable', 'unstable', 'stable', 'unstable',
       'unstable', 'unstable', 'stable', 'stable', 'unstable', 'stable',
       'unstable', 'stable', 'unstable', 'stable', 'stable', 'unstable',
       'unstable', 'stable', 'stable', 'unstable', 'stable', 'stable',
       'stable', 'unstable', 'stable', 'unstable', 'stable', 'stable',
       'stable', 'stable', 'stable', 'stable', 'unstable', 'stable',
       'stable', 'unstable', 'unstable', 'unstable', 'stable', 'stable',
       'unstable', 'unstable', 'unstable', 'unstable', 'unstable',
       'stable', 'stable', 'unstable', 'stable', 'stable', 'unstable',
       'stable', 'unstable', 'unstable', 'stable', 'unstable', 'unstable',
       'unstable', 'stable', 'unstable', 'stable', 'stable', 'unstable',
       'unstable', 'unstable', 'unstable', 'unstable', 'stable',
       'unstable', 'unstable', 'unstable', 'unstable', 'stable', 'stable',
    

In [44]:
from sklearn.ensemble import RandomForestClassifier
random_model = RandomForestClassifier(n_estimators=10, random_state = 1)
random_model.fit(normalised_train_df, y_balanced)

random_model.predict(normalised_test_df)

array(['stable', 'unstable', 'stable', 'unstable', 'unstable', 'unstable',
       'stable', 'stable', 'stable', 'stable', 'unstable', 'stable',
       'unstable', 'unstable', 'stable', 'unstable', 'stable', 'unstable',
       'stable', 'unstable', 'stable', 'unstable', 'unstable', 'stable',
       'stable', 'stable', 'stable', 'stable', 'unstable', 'stable',
       'unstable', 'stable', 'unstable', 'stable', 'stable', 'stable',
       'stable', 'stable', 'stable', 'unstable', 'stable', 'stable',
       'stable', 'unstable', 'unstable', 'stable', 'stable', 'unstable',
       'stable', 'unstable', 'unstable', 'stable', 'stable', 'stable',
       'unstable', 'stable', 'stable', 'stable', 'unstable', 'unstable',
       'stable', 'stable', 'unstable', 'stable', 'unstable', 'stable',
       'stable', 'stable', 'stable', 'unstable', 'unstable', 'unstable',
       'unstable', 'unstable', 'unstable', 'unstable', 'unstable',
       'unstable', 'unstable', 'stable', 'stable', 'stable', 'unstable'

In [45]:
from sklearn.ensemble import ExtraTreesClassifier
Extree_model=ExtraTreesClassifier(n_estimators=100, random_state=1)
Extree_model.fit(normalised_train_df, y_balanced)

Extree_model.predict(normalised_test_df)

array(['stable', 'unstable', 'stable', 'unstable', 'unstable', 'unstable',
       'stable', 'stable', 'stable', 'stable', 'unstable', 'unstable',
       'unstable', 'unstable', 'stable', 'unstable', 'stable', 'unstable',
       'stable', 'unstable', 'stable', 'unstable', 'unstable', 'stable',
       'stable', 'stable', 'unstable', 'stable', 'unstable', 'stable',
       'unstable', 'stable', 'unstable', 'stable', 'stable', 'stable',
       'stable', 'stable', 'stable', 'unstable', 'stable', 'stable',
       'unstable', 'unstable', 'unstable', 'stable', 'stable', 'unstable',
       'unstable', 'unstable', 'unstable', 'unstable', 'stable', 'stable',
       'unstable', 'stable', 'stable', 'stable', 'stable', 'unstable',
       'stable', 'stable', 'unstable', 'stable', 'unstable', 'stable',
       'unstable', 'stable', 'stable', 'unstable', 'unstable', 'unstable',
       'unstable', 'unstable', 'unstable', 'unstable', 'unstable',
       'unstable', 'unstable', 'stable', 'stable', 'stable', 

In [39]:
from sklearn.model_selection import RandomizedSearchCV

n_estimators = [50, 100, 300, 500, 1000]
#number of features to consider at every split
max_features = ['auto', 'sqrt', 'log2', None]

#minimum number of samples required at every split
min_samples_split = [2,3,5,7,9]

#minimum number of samples required at each leaf node
min_samples_leaf = [1,2,4,6,8]

#create hyper_parameter grid
hyper_parameter_grid = {'max_features': max_features, 
                        'min_samples_split': min_samples_split,  
                        'min_samples_leaf': min_samples_leaf}

model_tuned = RandomizedSearchCV(dec_tree, param_distributions=hyper_parameter_grid, random_state=1)

In [41]:
dd = model_tuned.fit(normalised_train_df, y_balanced)
dd.best_params_

{'min_samples_split': 7, 'min_samples_leaf': 6, 'max_features': None}

In [43]:
from sklearn.ensemble import GradientBoostingClassifier
grad_model = GradientBoostingClassifier(n_estimators=100, learning_rate = 1.0, max_depth = 1, random_state = 1)
grad_model.fit(normalised_train_df, y_balanced)

grad_model.predict(normalised_test_df)

array(['stable', 'unstable', 'unstable', 'unstable', 'unstable',
       'unstable', 'stable', 'unstable', 'unstable', 'stable', 'unstable',
       'unstable', 'unstable', 'unstable', 'stable', 'unstable', 'stable',
       'unstable', 'stable', 'unstable', 'stable', 'unstable', 'unstable',
       'stable', 'stable', 'stable', 'stable', 'stable', 'stable',
       'stable', 'unstable', 'stable', 'unstable', 'stable', 'unstable',
       'stable', 'stable', 'stable', 'stable', 'unstable', 'stable',
       'stable', 'unstable', 'unstable', 'stable', 'unstable', 'stable',
       'unstable', 'stable', 'unstable', 'unstable', 'stable', 'stable',
       'stable', 'stable', 'stable', 'stable', 'unstable', 'stable',
       'unstable', 'stable', 'stable', 'unstable', 'unstable', 'unstable',
       'stable', 'stable', 'stable', 'stable', 'unstable', 'unstable',
       'unstable', 'unstable', 'unstable', 'unstable', 'unstable',
       'unstable', 'unstable', 'unstable', 'stable', 'stable', 'stable',
